<img src="http://www.openfisca.fr/hotlinks/logo-openfisca.svg" width="198" align="right">
<h1 id="tocheading">Getting Started</h1>

This notebook illustrates the [key concepts](https://doc.openfisca.fr/en/key-concepts.html) of OpenFisca that you will find in the documentation.

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>


## Calculate a variable

Do some imports

In [79]:
from openfisca_france import FranceTaxBenefitSystem

Initialize the french [Tax and Benefit system](https://doc.openfisca.fr/en/tax_and_benefit_system.html) : it launches the entire legislation.

In [80]:
tax_benefit_system = FranceTaxBenefitSystem()

Create a [scenario](https://doc.openfisca.fr/en/input_data.html#scenario): first step to give a familial situation for which you want to calculate taxes or benefits.

In [81]:
scenario = tax_benefit_system.new_scenario()

Initialize a test case (the familial situation) in the scenario:
here we have one parent and two children.

In [82]:
scenario.init_single_entity(
    period = 2015,
    parent1 = dict(
        age = 30,
        salaire_de_base = 20000,
        ),
    enfants = [
        dict(age = 12),
        dict(age = 18),
        ],
    )

Create a [simulation](https://doc.openfisca.fr/en/simulation.html#simulation--the-framework-of-computation): the OpenFisca frame for calculating taxes or benefits.

In [83]:
simulation = scenario.new_simulation()

Calculate a variable : `"af"` (allocations familiales) for example.

In [84]:
simulation.calculate('af', '2015-01')

array([ 129.99000549], dtype=float32)

## Test the impact of a reform

*An reform represents a modified version of the real Tax and Benefit legislation* 

Reforms are gather in the module [`Reform`](https://github.com/openfisca/openfisca-france/tree/master/openfisca_france/reforms) of OpenFisca.

You have then to import the module extension corresponding to the reform of your concern.

Here we choose the fiscal reform of Landais, Piketty and Saez on the income tax (http://www.revolution-fiscale.fr/la-reforme-proposee). 

It is already implemented in OpenFisca under the name `landais_piketty_saez` .

In [85]:
from openfisca_france.reforms import landais_piketty_saez

Create a modified version of the tax and benefit system, affected by the changes introduced by the reform

In [86]:
reform = landais_piketty_saez.landais_piketty_saez(tax_benefit_system)

Create the scenario you want to test.

In [87]:
def init_profile(scenario):
    scenario.init_single_entity(
        period = '2013',
        parent1 = dict(
            age = 40,
            salaire_de_base = 50000,
            ),
        )
    return scenario

With the reform

In [88]:
#Indicate that you want to perfom the reform on this scenario
reform_scenario = init_profile(reform.new_scenario())

#Simulate the reform
reform_simulation = reform_scenario.new_simulation() 

# Choose the variable you want to calcul : here the disposable income, "revenu_disponible"
reform_simulation.calculate('revenu_disponible', '2013')

array([ 15614.34863281], dtype=float32)

Without the reform: the counterfactual

In [89]:
#Indicate that you want to perfom the standard system on this scenario
reference_scenario = init_profile(tax_benefit_system.new_scenario())

#Simulate the standard scenario
reference_simulation = reference_scenario.new_simulation()

# Choose the variable you want to calcul
reference_simulation.calculate('revenu_disponible', '2013')

array([ 36039.1875], dtype=float32)

## Trace the calculation of a variable

Thanks to the Trace tool you can see all the details of the variable calculation depending on the scenario you made.

In [90]:
from openfisca_core import tools
print(tools.get_trace_tool_link(scenario, variables = ['irpp'], api_url = 'http://api.openfisca.fr', trace_tool_url =  'http://www.openfisca.fr/tools/trace'))

http://www.openfisca.fr/tools/trace?api_url=http%3A%2F%2Fapi.openfisca.fr&simulation=%7B%22scenarios%22%3A+%5B%7B%22period%22%3A+%222015%22%2C+%22test_case%22%3A+%7B%22familles%22%3A+%5B%7B%22id%22%3A+0%2C+%22parents%22%3A+%5B%22ind0%22%5D%2C+%22enfants%22%3A+%5B%22ind2%22%2C+%22ind3%22%5D%7D%5D%2C+%22foyers_fiscaux%22%3A+%5B%7B%22id%22%3A+0%2C+%22declarants%22%3A+%5B%22ind0%22%5D%2C+%22personnes_a_charge%22%3A+%5B%22ind2%22%2C+%22ind3%22%5D%7D%5D%2C+%22individus%22%3A+%5B%7B%22id%22%3A+%22ind0%22%2C+%22age%22%3A+30%2C+%22salaire_de_base%22%3A+20000.0%7D%2C+%7B%22id%22%3A+%22ind2%22%2C+%22age%22%3A+12%7D%2C+%7B%22id%22%3A+%22ind3%22%2C+%22age%22%3A+18%7D%5D%2C+%22menages%22%3A+%5B%7B%22id%22%3A+0%2C+%22personne_de_reference%22%3A+%22ind0%22%2C+%22enfants%22%3A+%5B%22ind2%22%2C+%22ind3%22%5D%7D%5D%7D%7D%5D%2C+%22variables%22%3A+%5B%22irpp%22%5D%7D


In [3]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>